In [24]:
import numpy as np
import random
import torch
import pandas as pd
import copy

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# World Sense Disambiguation (WSD)

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées. L'objectif final est de comparer ces performances pour déterminer à partir de combien de données annotées la première méthode est préférable à la deuxième et vice versa.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)


In [25]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples qui seront utilisés lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (set) : corpus d'apprentissage
        self.X_test (set) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (set) : corpus de test
        self.y_test (set) : numéros des sens correspondant au corpus d'apprentissage
        self.annotated_examples (list) : liste des données considérées comme annotées 
        self.unannotated_examples (list) : liste des données considérées comme non annotées 
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = set()
        self.X_test = set()
        self.y_train = set()
        self.y_test = set()

        # liste des données considérées comme annotées 
        self.annotated_examples = []
        # liste des données considérées comme non annotées 
        self.unannotated_examples = []
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples représentatifs du corpus selon un nombre imposé de 
        données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de sens associés à ce mot
        sample_size (float) : quantité des données considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        
        # ajoute le nombre de données manquantes (non-présentes dans la liste) selectionnées au hasard
        random.shuffle(examples)
        for example in examples:
            if example not in selected_examples and len(selected_examples) < sample_size*len(examples):
                selected_examples.append(example)

        return selected_examples
    
    def select_train_test(self, x, y, senses, data_size, train_size = 0.8):
        '''
        Sépare en un train set et test set les examples représentatifs du corpus en mettant 
        au moins une étiquette de chaque sens dans le train set (et le test set si possible)

        INPUT
        x (list) : liste d'examples pour le mot à désambiguiser
        y (list) : liste d'étiquette pour chaque example de x
        senses (list) : numéros des sens associés à ce mot
        train_size (float) : quantité des données d'entraînement considérés
        '''

        # SELECTION DES DONNEES REPRESENTATIVES DU TRAIN

        # sélection au hasard d'un contexte et de son étiquette pour chaque sens
        selected_train = [random.choice(list(filter((lambda example: example[1] == sense),
                                                    zip(x,y)))) for sense in senses]
            
        # SELECTION DES DONNEES REPRESENTATIVES DU TEST

        # fait de même pour le corpus de test, lorsque possible, en ne considérant pas
        # ceux déjà sélectionnés pour le train
        selected_test = []
        for sense in senses:
            added = False
            for example in zip(x,y):
                for train_ex in selected_train:
                    # vérifie que l'example n'a pas déjà été sélectionné dans le train et
                    # le test pour ne pas faire de doublon
                    if example[1] == sense and example[1] == train_ex[1] and not(np.array_equal(example[0], train_ex[0])):
                        selected_test.append(example)
                        added = True
                        break
                if added == True:
                    break

        # SELECTION DES DONNEES RESTANTES DU TRAIN ET DU TEST

        # effectue un split sur le reste d'examples disponibles
        remaining_X = []
        remaining_y = []
        selected_train_copy = copy.deepcopy(selected_train)
        selected_train_copy.extend(selected_test)
        for contexte, sens in zip(x,y):
            to_add = True
            for ex in selected_train_copy:
                # vérifie que l'example n'a pas déjà été sélectionné dans le train et le
                # test pour ne pas faire de doublon
                if np.array_equal(contexte, ex[0]) and sens == ex[1]:
                    to_add = False
                    break
            if to_add:
                remaining_X.append(contexte)
                remaining_y.append(sens)

        try:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(remaining_X, remaining_y,train_size = train_size)
    
            # ajout des données représentatives dans les sets correspondant
            selected_X_train, selected_y_train = zip(*selected_train)
            selected_X_test, selected_y_test = zip(*selected_test)
            
            self.X_train.extend(selected_X_train)
            self.X_test.extend(selected_X_test)
            self.y_train.extend(selected_y_train)
            self.y_test.extend(selected_y_test)
            
        except:
            print(f"La taille des données ({data_size}) est trop faible. L'opération train_test_split() est impossible.")                                                                     
            self.X_train, self.X_test, self.y_train, self.y_test = set(), set(), set(), set()
                     
    def define_instance(self, instance, data_size, display=True):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        affichage (bool, optional): affiche une trace des options choisies
        '''
        ###################################################
        # Etape 1: faire le set de données annotées
        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.annotated_examples = self.select_examples(self.w2examples[instance],
                                                       self.w2senses[instance],
                                                       data_size)
        # liste de tous les contextes sous forme d'embedding
        X_annotated = [self.look_up(context) for context, gold in self.annotated_examples]
        # liste de tous les numéros de sens
        y_annotated = [gold for context, gold in self.annotated_examples]
        # le set des données annotées est une liste des examples représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(X_annotated,y_annotated))

        ###################################################
        # Etape 2: faire le set de données non-annotées
        # garde en mémoire les examples considérés comme non-annotés
        self.unannotated_examples = [example for example in self.w2examples[instance] 
                                     if example not in self.annotated_examples]        
        # liste de tous les contextes sous forme d'embedding
        X_unannotated = [self.look_up(context) for context,gold in self.unannotated_examples]
        # liste de tous les numéros de sens
        y_unannotated = [gold for context,gold in self.unannotated_examples]
        # le set des données non-annotées est une liste des examples sous forme d'embeddings
        self.unannotated_set = list(zip(X_unannotated,y_unannotated))

        ###################################################
        # Etape 3: faire le set de données à classifier par MLPClassifier_WSD
        # création des corpus d'entrainement et de test
        self.select_train_test(X_annotated, y_annotated, self.w2senses[instance],data_size)
        
        ###################################################
        # Etape 4: affichage des informations relatives à l'instance choisie pour MLPClassifier_WSD
        if display :
            self.classifier_display_at_instance_time(instance, data_size)
            
    
    # Affichage des informations relatives à l'instance choisie pour MLPClassifier_WSD
    def classifier_display_at_instance_time(self, instance, data_size):
        print("--> Instance :", instance)
        print(f'{data_size*100}% des données annotées considérées')
        print("Nombre de données d'entraînement : ", len(self.X_train))
        print("Etiquettes possibles pour cette instance : ", self.w2senses[instance])
        if self.y_train:
            print("Etiquettes présentes dans les données d'entraînement :", Counter(self.y_train))
            most_frequent_sense = max(self.y_train,key=self.y_train.count)
            occurrence_of_most_frequent_sense = self.y_train.count(most_frequent_sense)/len(self.y_train)
            #print(f"Les occurrences du sens {most_frequent_sense} sont les plus fréquentes, avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
    
    # Affichage des informations relatives à l'instance choisie pour K_Means
    def kmeans_display_at_instance_time(self, instance, num_correct, num_examples, size):
        y = [gold for tensor,gold in self.selected_examples_embs] + [gold for tensor,gold in self.unannotated_set] 
        print("--> Instance :",instance)
        print(f"Proportion de données annotées considérées sur le corpus: {size*100} %")
        print(f"Répartition des sens: {Counter(y)}")
        most_frequent_sense = max(y,key=y.count)
        occurrence_of_most_frequent_sense = y.count(most_frequent_sense)/len(y)
        print(f"Le sens le plus fréquent pour '{instance}' est le sens {most_frequent_sense} avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
        #print(f"Pour '{instance}', l'accuracy est de {round(num_correct/num_examples,2)} %")
        print()

    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_annotated_and_unannotated_sets(self):
        
        return self.selected_examples_embs, self.unannotated_set
        
    def get_annotated_examples(self):
        return self.annotated_examples
    
    def get_unannotated_examples(self):
        return self.unannotated_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [26]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "../donnees/embeddings.txt"

In [27]:
# exemple d'utilisation d'Extractor avec le verbe "aboutir" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("aboutir", 0.1, display=True)
print("\nexemple d'example avec contexte et gold class :",ext.get_annotated_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])

La taille des données (0.1) est trop faible. L'opération train_test_split() est impossible.
--> Instance : aboutir
10.0% des données annotées considérées
Nombre de données d'entraînement :  0
Etiquettes possibles pour cette instance :  {'1', '4', '2', '3'}

exemple d'example avec contexte et gold class : (['quai', 'de', 'grenelle', 'et', 'aboutir', 'rue', 'de', 'la', 'croix'], '1')

au format embedding :  (array([ 1.1260e-01,  1.1970e-01,  6.5350e-01, -6.0890e-01, -1.0010e-01,
       -1.3570e-01,  7.1280e-01, -2.0900e-02, -2.0390e-01, -6.1000e-01,
       -1.9460e-01, -3.1340e-01,  1.1800e-01,  5.8740e-01, -8.3500e-02,
        1.8990e-01,  5.5980e-01, -2.1220e-01, -4.5650e-01,  1.0880e-01,
        1.7000e-03, -4.7080e-01, -4.1750e-01,  3.7060e-01,  1.4750e-01,
        1.6180e-01,  1.1130e-01, -2.0490e-01,  3.9200e-01,  2.4550e-01,
        1.0200e-01, -2.3870e-01,  1.4240e-01, -2.2780e-01,  3.0380e-01,
        3.9980e-01, -7.0890e-01,  3.6210e-01,  3.1320e-01,  8.0000e-02,
       -4.2190


## Classification supervisée

L'utilisation d'un Classifieur MLP permet de prédire le sens des verbes de façon supervisée. Il s'agit de la première méthode proposée pour la tâche de WSD.

Les données utilisées sont les données annotées extraites du fichier FSE grâce à l'Extractor pour un lemme donné. Un MLPClassifier_WSD doit donc être créé pour chaque verbe dont on veut la prédiction.

MLPClassifier_WSD comporte les méthodes suivantes :
- `classify()` : pour prédire le sens du lemme associé au classifieur et calculer l'accuracy de cette prédiction


In [28]:
class MLPClassifier_WSD :
    
    def __init__(self, X_train, X_test, y_train, y_test) :
        '''
        A partir des données, prédit le sens d'un lemme et évalue la prédiction

        ARGUMENTS DE CLASSE
        self.X_train (list) : liste des contextes d'entraînement
        self.y_train (list) : liste des gold class d'entraînement
        self.X_test (list) : liste des contextes de test
        self.y_test (list) : liste des gold class de test
        '''
        # ensembles d'entraînement et de test provenant de l'Extractor pour un lemme donné
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 

        # prédiction du modèle
        self.y_pred = []
        # accuracy de la prédiction
        self.accuracy = 0
        
    def classify(self, display=False) :
        ''' 
        Permet de prédire le sens du verbe.

        INPUT
        display (bool, optional) : affiche une trace des options choisies
        '''
        try:
            clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,)) 
            # entraînement
            clf.fit(self.X_train, self.y_train)
            # prédiction
            self.y_pred = clf.predict(self.X_test)

            # calcul de l'accuracy
            self.accuracy = accuracy_score(self.y_pred, self.y_test)
            
            # si une trace est souhaitée
            if display :
                print("prediction :", self.y_pred)
                print("gold :", self.y_test)
                print("accuracy score : ", round(self.accuracy,2),"\n")
        except:
            print("ATTENTION: La quantité de données fournies est insuffisante: formation du test set impossible")
        

In [29]:
# exemple d'utilisation de MLPClassifieur_WSD avec le verbe "demeurer" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("demeurer", 0.1, display=True)
X_train, X_test, y_train, y_test = ext.get_sets()

Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
Clf.classify(display=True)

La taille des données (0.1) est trop faible. L'opération train_test_split() est impossible.
--> Instance : demeurer
10.0% des données annotées considérées
Nombre de données d'entraînement :  0
Etiquettes possibles pour cette instance :  {'4', '6', '2', '3'}
ATTENTION: La quantité de données fournies est insuffisante: formation du test set impossible


Pour pouvoir comparer les performances de ce Classifieur MLP avec celles de l'apprentissage semi-supervisé, nous devons créer un classifieur par verbe à désambiguiser et choisir un pas de descente pour la quantité de données annotées considérées.

Pour chaque classifieur et chaque quantitée de données considérées, on effectue *n_repeat* classifications pour obtenir une accuracy moyenne représentative du classifieur. 

Par conséquent, pour :
- n_repeat = 5
- step = 0.25

nous obtiendrons pour chaque classifieur une liste d'accuracies correspondant à :
- la moyenne des accuracies de 5 prédictions 
- pour 100%, 75%, 50% et 25% des données.

In [30]:
def get_mean_accuracy(instance, data_size, n_repeat, affichage=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    affichage (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean (float): moyenne des accuracies calculées
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
        
    for n in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        X_train, X_test, y_train, y_test = ext.get_sets()
        
        if y_train :
            Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
            Clf.classify()
                
            accuracies.append(Clf.accuracy)
            mean = sum(accuracies)/len(accuracies)
            
        else :
            accuracies.append("impossible")
            mean = "impossible"
            
    # si la trace est souhaitée
    if affichage :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", mean, "\n")

    return mean

def get_accuracies(instances, step, n_repeat, affichage=True):
    '''
    - Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité 
      de données considérée.
    - Produit un fichier csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    moyennes = []
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 1.0 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de son accuracy moyenne
            instance2acc[instance].append(get_mean_accuracy(instance, data_size, n_repeat, affichage))

        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        accuracies = [accuracies[i] for accuracies in instance2acc.values() if accuracies[i] != "impossible"]
        moyennes.append(sum(accuracies)/len(accuracies))
  
    # ajout d'une clé moyennes pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    instance2acc["moyennes"] = moyennes
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes 

    # export en csv des résultats
    df = pd.DataFrame(instance2acc)
    df.set_index("data_sizes")
    df.to_csv(f"accuracies_par_lemme_{n_repeat}_repet.csv")
    
    return instance2acc     

In [31]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5

In [32]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des accuracies
get_accuracies(instances, step, n_repeat)

100.0% des données annotées considérées
instance : aboutir
accuracies : [0.8181818181818182, 0.9090909090909091, 0.9090909090909091, 1.0, 0.8181818181818182]
accuracy moyenne : 0.8909090909090909 

100.0% des données annotées considérées
instance : investir
accuracies : [0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.4166666666666667]
accuracy moyenne : 0.5666666666666667 

100.0% des données annotées considérées
instance : traduire
accuracies : [0.6666666666666666, 0.9166666666666666, 0.8333333333333334, 0.5, 0.9166666666666666]
accuracy moyenne : 0.7666666666666666 

100.0% des données annotées considérées
instance : témoigner
accuracies : [0.8333333333333334, 0.8333333333333334, 0.9166666666666666, 1.0, 0.9166666666666666]
accuracy moyenne : 0.9 

100.0% des données annotées considérées
instance : juger
accuracies : [0.6, 0.8, 0.7, 0.7, 0.5]
accuracy moyenne : 0.6599999999999999 



c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

100.0% des données annotées considérées
instance : justifier
accuracies : [0.7272727272727273, 0.7272727272727273, 0.8181818181818182, 0.8181818181818182, 0.9090909090909091]
accuracy moyenne : 0.8 

100.0% des données annotées considérées
instance : viser
accuracies : [1.0, 1.0, 1.0, 1.0, 1.0]
accuracy moyenne : 1.0 



c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

100.0% des données annotées considérées
instance : prononcer
accuracies : [0.38461538461538464, 0.6153846153846154, 0.3076923076923077, 0.38461538461538464, 0.5384615384615384]
accuracy moyenne : 0.4461538461538462 

100.0% des données annotées considérées
instance : accomplir
accuracies : [0.6153846153846154, 0.6923076923076923, 0.6923076923076923, 0.6153846153846154, 0.6923076923076923]
accuracy moyenne : 0.6615384615384615 



...

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


50.0% des données annotées considérées
instance : arriver
accuracies : [0.5, 0.25, 0.625, 0.5, 0.25]
accuracy moyenne : 0.425 

50.0% des données annotées considérées
instance : chercher
accuracies : [0.8571428571428571, 0.7142857142857143, 0.42857142857142855, 0.5714285714285714, 1.0]
accuracy moyenne : 0.7142857142857142 

50.0% des données annotées considérées
instance : composer
accuracies : [1.0, 1.0, 1.0, 0.8333333333333334, 1.0]
accuracy moyenne : 0.9666666666666668 



c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


50.0% des données annotées considérées
instance : confier
accuracies : [1.0, 0.7142857142857143, 1.0, 0.5714285714285714, 0.8571428571428571]
accuracy moyenne : 0.8285714285714285 

25.0% des données annotées considérées
instance : aboutir
accuracies : [0.75, 1.0, 0.5, 1.0, 1.0]
accuracy moyenne : 0.85 

25.0% des données annotées considérées
instance : investir
accuracies : [0.4, 0.0, 0.4, 0.25, 0.6]
accuracy moyenne : 0.32999999999999996 

25.0% des données annotées considérées
instance : traduire
accuracies : [0.6, 0.4, 0.6, 0.6, 0.8]
accuracy moyenne : 0.6 

25.0% des données annotées considérées
instance : témoigner
accuracies : [0.75, 0.75, 0.5, 0.75, 1.0]
accuracy moyenne : 0.75 

25.0% des données annotées considérées
instance : juger
accuracies : [0.75, 0.6666666666666666, 1.0, 1.0, 1.0]
accuracy moyenne : 0.8833333333333332 

25.0% des données annotées considérées
instance : justifier
accuracies : [1.0, 0.75, 0.6666666666666666, 0.5, 0.5]
accuracy moyenne : 0.6833333333333333

{'aboutir': [0.8909090909090909, 0.7333333333333333, 0.8, 0.85],
 'investir': [0.5666666666666667,
  0.6000000000000001,
  0.5666666666666667,
  0.32999999999999996],
 'traduire': [0.7666666666666666, 0.78, 0.7428571428571429, 0.6],
 'témoigner': [0.9, 0.8222222222222222, 0.8285714285714286, 0.75],
 'juger': [0.6599999999999999, 0.65, 0.6133333333333334, 0.8833333333333332],
 'justifier': [0.8, 0.8222222222222222, 0.7, 0.6833333333333333],
 'viser': [1.0, 0.9555555555555555, 0.9142857142857143, 0.85],
 'prononcer': [0.4461538461538462,
  0.41818181818181815,
  0.4,
  0.47000000000000003],
 'accomplir': [0.6615384615384615,
  0.6872727272727273,
  0.6107142857142858,
  0.77],
 'convenir': [0.6, 0.5636363636363637, 0.375, 0.51],
 'acquérir': [0.5142857142857142, 0.5927272727272727, 0.621031746031746, 0.5],
 'achever': [0.8545454545454547, 0.8666666666666666, 0.7666666666666667, 0.5],
 'observer': [0.5846153846153845,
  0.54,
  0.6107142857142858,
  0.5399999999999999],
 'adapter': [0.569

## Classification semi-supervisée

In [33]:
import numpy as np
import torch
import pandas as pd
import random
from collections import Counter

class K_Means():
    ''' 
    classifieur K-means pour un mot particulier
    '''

    def __init__(self, annotated_examples, not_annotated_examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        examples : liste d'examples dont le mot à désambiguiser est le même pour 
                   chaque example
        example : couple d'un mot avec son contexte de fenêtre 4 (sous forme 
                  d'embedding) et du numéro de sens attendu du mot à désambiguiser 
                  (gold class sous forme d'integer)
                    si example = ([1.9, 2.3, 0.6], 1),
                    - le contexte avec le mot à désambiguiser et son lemme est 
                      l'embedding [1.9, 2.3, 0.6]
                    - le numéro de sens est 1
        '''

        # transforme l'ensemble des examples en une liste pour pouvoir garder le 
        # même indice pour chaque example par la suite
        self.annotated_examples = annotated_examples

        # on a le gold, mais on ne l'utilisera que pour calculer l'accuracy
        # pour le training, nous ferons comme si nous n'avions pas la gold class
        self.not_annotated_examples = not_annotated_examples

        # transforme les embeddings en tensors
        # ce sont les exemples qui devront être classifiés
        self.tensors_examples = [example[0] for example in self.not_annotated_examples]

        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données annotées, qui représentent tous les sens possibles
        self.k = len(set([example[1] for example in self.annotated_examples]))
        
        # initialisation des centroïdes : pour chaque sens, le centroïde 
        # correspond à la moyenne des embeddings des exemples annotés
        # Ainsi, chaque centroïde représente un sens
        self.tensors_centroids, self.cluster2sense = self.make_centroids()

        #initialisation d'un dataframe à partir de self.tensors_centroids
        # rows: cluster number (k), columns: dimension number (len(tensors_examples[0]))
        self.new_centroids = pd.DataFrame(self.tensors_centroids).astype("float")
        #print(self.new_centroids.head())
        # initialisation de clusters : tous les examples sont associés au cluster 0
        self.clusters = np.zeros(len(not_annotated_examples))
        #random number between 0 and k-1 for each example
        #self.clusters = np.asarray([random.randint(0,self.k-1) for i in range(len(not_annotated_examples))])


    def make_centroids(self):
        cluster2sense = []
        tensors_centroids = []
        senses = set([example[1] for example in self.annotated_examples])
        for sense in senses: 
            # on récupère les examples du sens
            examples_sense = [example[0] for example in self.annotated_examples if example[1] == sense]
            # on calcule le centroid du sens
            centroid = torch.mean(torch.stack(examples_sense), dim=0)
            # on ajoute le centroid à la liste des centroids
            tensors_centroids.append(centroid)
            # on ajoute le sens à la liste des sens
            #L'index du sens correspond au numéro du cluster
            cluster2sense.append(sense)

        return tensors_centroids, cluster2sense
    

    def update_new_centroids_dataframe(self):
        # étape 1: on regroupe tous les tenseurs des exemples ayant le même cluster dans self.clusters et on fait la moyenne
        tmp_new_centroids = pd.DataFrame(self.tensors_examples).groupby(by = self.clusters).mean() #shape : [k, len(tensors_examples[0])]
        # étape 2: on détermine quels sont les clusters qui ont été assignés à au moins un exemple dans self.clusters
        clus, counts = np.unique(self.clusters, return_counts=True)
        #print(dict(zip(clus, counts)))
        # étape 3: pour chaque cluster qui n'a pas d'exemple assigné, on insère un row vide dans tmp_new_centroids à l'index correspondant
        row_to_insert = [[np.nan] * self.new_centroids.shape[1]]
        for i in range(self.k): 
            if i not in clus:
                tmp_new_centroids = pd.concat([tmp_new_centroids.iloc[:i], pd.DataFrame(row_to_insert, columns=tmp_new_centroids.columns), tmp_new_centroids.iloc[i:]]).reset_index(drop=True)
        # étape 4: on remplace dans self.new_centroids les rows non vides de tmp_new_centroids 
        self.new_centroids.update(tmp_new_centroids)


    def learn_clusters(self):
        '''
        Algorithme de K-Means
        Retourne les coordonnées de chaque centroide ainsi que le cluster auquel 
        appartient chaque example
        '''

        # différence initialisée à Vrai
        diff = True
        
        # tant qu'il y a une différence entre l'ancienne liste et la nouvelle 
        # liste de centroides
        #while diff:
        for i in range(10):

            # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE
         
            # pour chaque couple (indice, coordonnées) dans les examples
            for i, tensor_example in enumerate(self.tensors_examples):
                # initialisation de la distance minimum à l'infini
                min_dist = float('inf')
                # pour chaque couple (indice, coordonnées) dans les centroides
                for j, tensor_centroid in enumerate(self.tensors_centroids):
                    # calcul de la distance entre cet example et ce centroide
                    d = (tensor_centroid - tensor_example).pow(2).sum(axis=0).sqrt()
                    # si une distance plus faible est trouvée
                    if min_dist > d:
                        # la distance ainsi que le centroide sont stockés
                        min_dist = d
                        self.clusters[i] = j
            

            #mise à jour des centroïdes: 
            self.update_new_centroids_dataframe()
            
            tensors_new_centroids = []
            
            #pour chaque cluster (donc chaque ligne de new_centroids)
            for i in range(len(self.new_centroids.index)):
                centroid = []
                #pour chaque dimension du vecteur moyen assigné au cluster
                for j in range(len(self.new_centroids.columns)):
                    # recréee le vecteur
                    centroid.append(int(self.new_centroids.iat[i,j]))
                #et rassemble les nouveaux tenseurs des centroides dans une liste
                tensors_new_centroids.append(torch.tensor(centroid))

            # MISE A JOUR DES CENTROIDES
            count_diff = 0
            # pour chaque centroide
            for i in range(len(self.tensors_centroids)):
                # si l'ancien centroide et le nouveau ne sont pas les mêmes
                if not(torch.equal(self.tensors_centroids[i], tensors_new_centroids[i])):
                    count_diff += 1
                    # met à jour le centroide
                    self.tensors_centroids = tensors_new_centroids
            # s'il n'y a eu aucune différence entre les anciens et les nouveaux centroides, 
            # la boucle while se termine
            if count_diff == 0:
                diff = False
            
            

    def accuracy(self): 
        correct = 0
        for i in range(len(self.not_annotated_examples)):
            sens_attendu = self.not_annotated_examples[i][1]
            sens_assigne = self.cluster2sense[int(self.clusters[i])]
            if sens_attendu == sens_assigne:
                correct += 1
        return correct, len(self.not_annotated_examples)

In [34]:
def get_mean_accuracy(instance, data_size, n_repeat, affichage=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    affichage (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean (float): moyenne des accuracies calculées
    '''

    # liste de l'accuracy à chaque répétition


    accuracies = []
    accuracy_instances = []
    list_num_correct = []
    list_num_examples = []
        
    for _ in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        
        annotated_set, unannotated_set = ext.get_annotated_and_unannotated_sets()
        annotated_set = [(torch.from_numpy(embedding),sense) for embedding,sense in annotated_set]
        unannotated_set = [(torch.from_numpy(embedding),gold) for embedding,gold in unannotated_set]
        k_Means = K_Means(annotated_set, unannotated_set)
        k_Means.learn_clusters()
        
        correct, num_examples = k_Means.accuracy()
        
        if _ == n_repeat-1:
            ext.kmeans_display_at_instance_time(instance, correct, num_examples, data_size)
        accuracies.append(correct/num_examples)
        mean = sum(accuracies)/len(accuracies)
        
        list_num_correct.append(correct)
        list_num_examples.append(num_examples)
        accuracy_instances.append(correct/num_examples)

        # si la trace est souhaitée
        if affichage and _ == n_repeat-1:
            print("Accuracies :", accuracies)
            print(f"Accuracy moyenne sur {n_repeat} répétitions:", mean)
            print()
    accuracy_kmeans = sum(accuracy_instances)/len(accuracy_instances)
    print(f"Macro-average : {round(accuracy_kmeans,2)}") #une accuracy par sens et tu divises par le nb de sens
    micro_average = sum(list_num_correct)/sum(list_num_examples)
    print(f"Micro-average : {round(micro_average,2)}")
    print("\n--------------------------------------------------\n")
    return mean

def get_accuracies(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité 
      de données considérée.
    - Produit un fichier csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    moyennes = []
    data_sizes =[]
    

    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 0.75 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de son accuracy moyenne
            mean = get_mean_accuracy(instance, data_size, n_repeat, display)
            instance2acc[instance].append(mean)
        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        moyennes.append(sum([accuracies[i] for accuracies in instance2acc.values()])/len(instance2acc))
  
    # ajout d'une clé moyennes pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    instance2acc["moyennes"] = moyennes
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes 

    # export en csv des résultats
    df = pd.DataFrame(instance2acc)
    df.set_index("data_sizes")
    df.to_csv(f"kmeans_accuracies_par_lemme_{n_repeat}_repet.csv")
    
    return instance2acc  

In [35]:
# pas de descente dans la quantité de données considérées
step = 0.15
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5

In [36]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())[:3]
# calcul des accuracies
get_accuracies(instances, step, n_repeat)

--> Instance : aboutir
Proportion de données annotées considérées sur le corpus: 75.0 %
Répartition des sens: Counter({'3': 41, '4': 7, '1': 1, '2': 1})
Le sens le plus fréquent pour 'aboutir' est le sens 3 avec une proportion de 82.0 %

Accuracies : [0.08333333333333333, 0.08333333333333333, 0.25, 0.5833333333333334, 0.08333333333333333]
Accuracy moyenne sur 5 répétitions: 0.21666666666666665

Macro-average : 0.22
Micro-average : 0.22

--------------------------------------------------

--> Instance : investir
Proportion de données annotées considérées sur le corpus: 75.0 %
Répartition des sens: Counter({'4': 34, '3': 11, '1': 4, '2': 1})
Le sens le plus fréquent pour 'investir' est le sens 4 avec une proportion de 68.0 %

Accuracies : [0.0, 0.08333333333333333, 0.0, 0.16666666666666666, 0.08333333333333333]
Accuracy moyenne sur 5 répétitions: 0.06666666666666667

Macro-average : 0.07
Micro-average : 0.07

--------------------------------------------------

--> Instance : traduire
Pro

{'aboutir': [0.21666666666666665,
  0.25,
  0.20740740740740743,
  0.16470588235294117,
  0.28095238095238095,
  0.07391304347826087,
  0.2],
 'investir': [0.06666666666666667,
  0.05,
  0.037037037037037035,
  0.011764705882352941,
  0.047619047619047616,
  0.06086956521739131,
  0.043478260869565216],
 'traduire': [0.3,
  0.35,
  0.4444444444444445,
  0.4,
  0.45714285714285713,
  0.4553191489361702,
  0.31489361702127655],
 'moyennes': [0.19444444444444442,
  0.21666666666666665,
  0.22962962962962963,
  0.19215686274509802,
  0.2619047619047619,
  0.19670058587727413,
  0.18612395929694725],
 'data_sizes': [0.75,
  0.6,
  0.45,
  0.30000000000000004,
  0.15000000000000002,
  0.0,
  -0.1499999999999999]}